First, we import the required packages.

In [6]:
## packages
import pandas as pd
import datetime
import time
import sys
import string
import re
import gc
import requests
import xml.etree.ElementTree as ET
from urllib.parse import urlencode
from urllib.request import urlopen
from urllib.error import HTTPError

Now, let's specify the parameters as though we were going to pass them to the arxivsearch function.

In [7]:
## params
start_date = str(datetime.date.today().replace(day=1))
end_date = str(datetime.date.today().replace(day=10))
kwd_req = ['online'] 
kwd_exc = ['education'] 
kwd_one = [['bandit', 'partial information'], ['regret', 'generalization']]
athr_req = [] 
athr_exc = [] 
athr_one = []
subject = 'stat' # subjects = ['stat']
max_records = 50
max_time = 300
cols = ['id', 'title', 'authors', 'date', 'categories', 'abstract']
export = '/Users/blairbilodeau/Desktop/arxiv/'
exportfile = ''
download = '/Users/blairbilodeau/Desktop/arxiv/'

We try accessing the url, which we print.

In [28]:
# set url to extract papers from this subject
OAI = '{http://www.openarchives.org/OAI/2.0/}'
ARXIV = '{http://arxiv.org/OAI/arXiv/}'
BASE = 'http://export.arxiv.org/oai2?verb=ListRecords&'

url = BASE + 'from=' + start_date + '&until=' + end_date + '&metadataPrefix=arXiv&set=%s' %subject

## error handling for opening and closing URL
try:
	#url_response = urlopen(url)
	#xml = url_response.read() # get raw xml data from server
	gc.collect()
except HTTPError as e:
	# handle service unavailable error for requesting too often
	if e.code == 503:
		retry_time = int(e.hdrs.get('retry-after', 30))
		print('Got 503. Retry after {0:d} seconds.'.format(retry_time))
	# if it's a different error, just have to raise it
	else:
		raise

In [29]:
print(url)

http://export.arxiv.org/oai2?verb=ListRecords&from=2020-05-01&until=2020-05-10&metadataPrefix=arXiv&set=stat


In [30]:
'http://from=' + start_date + '&until=' + end_date

'http://from=2020-05-01&until=2020-05-10'

In [31]:
print(xml[0:200])

1280168

Let's visualize a bit how ugly XML is for humans, and start extracting elements from it we can work with.

In [ ]:
xml_root = ET.fromstring(xml) # get root of xml hierarchy
print(xml_root)

In [ ]:
records = xml_root.findall(OAI + 'ListRecords/' + OAI + 'record') # list of all records from xml tree
print('We have fetched {0:d} records. Here is a sample.'.format(len(records)))
print(records[0:3])

In [ ]:
## extract metadata for each record
metadata = [record.find(OAI + 'metadata').find(ARXIV + 'arXiv') for record in records]
print(metadata[0:3])

We will use Python's list comprehension to extract the data. Here's an example of how that works.

In [33]:
## list comprehension
x = []
for i in range(10):
	x = x + [i]
print('x = ',x)

# v.s.

y = [i for i in range(10)]
print('y = ',y)

x=  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
y=  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
